# Домашнее задание №8. Рекуррентные нейронные сети RNN LSTM GRU

In [114]:
import pandas as pd
from sklearn.model_selection import train_test_split

from string import punctuation
from stop_words import get_stop_words
from pymorphy2 import MorphAnalyzer
import re
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf

from keras.losses import SparseCategoricalCrossentropy
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Input, Embedding, Conv1D, GlobalMaxPool1D, SimpleRNN, LSTM, GRU, Masking
from keras.callbacks import TensorBoard 
from keras.callbacks import EarlyStopping 

In [115]:
#функция подсчета f1_score
import keras.backend as K
def get_f1(y_true, y_pred): #taken from old keras source code
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    f1_val = 2*(precision*recall)/(precision+recall+K.epsilon())
    return f1_val

## Заданиe
Данные берем отызывы за лето

На вебинаре мы говорили, что долгое время CNN и RNN архитектуры были конурируещими. Выяснить, какая архитектура больше подходит для нашей задачи:
1. построить свёрточные архитектуры
2. построить различные архитектуры с RNN
3. построить совместные архитектуры CNN -> RNN или (RNN -> CNN)

## План решения

[0. Загрузка и просмотр данных](#section_0)

[1. Предобработка данных](#section_1)

[2. Сверточные архитектуры](#section_2)

[3. Рекуррентные архитектуры](#section_3)

[3.1. SimpleRNN](#section_3.1)

[3.2. LSTM](#section_3.2)

[3.3. GRU](#section_3.3)

[4. Совместные архитектуры](#section_4)

[4.1. CNN --> RNN](#section_4.1)

[4.2. RNN --> CNN](#section_4.2)

[5. Выводы](#section_5)

## Загрузка и просмотр данных  <a id='section_0'></a>

In [116]:
df = pd.read_excel('отзывы за лето.xls')
df.head()

Rating                                            Content        Date
0       5                                     It just works!  2017-08-14
1       4  В целом удобноное приложение...из минусов хотя...  2017-08-14
2       5                                        Отлично все  2017-08-14
3       5  Стал зависать на 1% работы антивируса. Дальше ...  2017-08-14
4       5                     Очень удобно, работает быстро.  2017-08-14

In [117]:
df['Rating'].value_counts()

5    14586
1     2276
4     2138
3      911
2      748
Name: Rating, dtype: int64

In [118]:
df_train, df_test = train_test_split(df, test_size=0.33, random_state=42)
df_train = df_train.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)
df_train.shape, df_test.shape

((13841, 3), (6818, 3))

In [119]:
df_train.head()

Rating                                            Content        Date
0       5  Наконец-то исправили эту чушь с неоргинальной ...  2017-08-09
1       5                             Удобно в использовании  2017-07-27
2       5                                            Отлично  2017-08-08
3       5                                              Класс  2017-07-25
4       5                                             Удобно  2017-07-08

## 1. Предобработка данных <a id='section_1'></a>

In [120]:
sw = set(get_stop_words("ru"))
exclude = set(punctuation)
morpher = MorphAnalyzer()

def preprocess_text(txt):
    txt = str(txt)
    txt = "".join(c for c in txt if c not in exclude)
    txt = txt.lower()
    txt = re.sub("\sне", "не", txt)
    txt = [morpher.parse(word)[0].normal_form for word in txt.split() if word not in sw]
    return " ".join(txt)

df_train['Content'] = df_train['Content'].apply(preprocess_text)
df_test['Content'] = df_test['Content'].apply(preprocess_text)

In [121]:
df_train.head()

Rating                                            Content        Date
0       5  наконецтый исправить чушь снеоргинальный проши...  2017-08-09
1       5                               удобно использование  2017-07-27
2       5                                            отлично  2017-08-08
3       5                                              класс  2017-07-25
4       5                                             удобно  2017-07-08

**Готовим словарь**

In [122]:
text_corpus_train = df_train['Content'].values
text_corpus_test = df_test['Content'].values
len(text_corpus_train), text_corpus_train #список текстов

(13841,
 array(['наконецтый исправить чушь снеоргинальный прошивка приложение удобно пользоваться',
        'удобно использование', 'отлично', ...,
        'мочь скачать ошибка номер 24', 'сбербанк', 'целое отлично'],
       dtype=object))

In [123]:
#векторизируем текстовый корпус, превращая каждый текст в последовательность целых чисел (индексы токенов в словаре)
tokenizer = Tokenizer(num_words=None, 
                     filters='#$%&()*+-<=>@[\\]^_`{|}~\t\n', #символы для удаления из текстов
                     lower = False, #перевод к нижнему регистру
                      split = ' ') #разделитель для слов

#обновляем внутренний словарь на основе списка токенов (используется перед text_to_sequences)
tokenizer.fit_on_texts(text_corpus_train)

#словарь из токенов
len(tokenizer.index_word), tokenizer.index_word

(10292,
 {1: 'приложение',
  2: 'удобно',
  3: 'работать',
  4: 'удобный',
  5: 'отлично',
  6: 'нравиться',
  7: 'хороший',
  8: 'отличный',
  9: 'телефон',
  10: 'супер',
  11: 'быстро',
  12: 'обновление',
  13: 'пароль',
  14: 'мочь',
  15: 'пользоваться',
  16: 'антивирус',
  17: 'банк',
  18: 'вход',
  19: 'устраивать',
  20: 'сбербанк',
  21: 'раз',
  22: 'прошивка',
  23: 'карта',
  24: 'проблема',
  25: 'рута',
  26: 'программа',
  27: 'ошибка',
  28: 'разработчик',
  29: 'сделать',
  30: 'приходиться',
  31: 'вводить',
  32: 'перевод',
  33: 'счёт',
  34: 'писать',
  35: 'норма',
  36: 'деньга',
  37: 'довольный',
  38: 'около',
  39: 'постоянно',
  40: 'нормально',
  41: 'код',
  42: 'исправить',
  43: 'смс',
  44: 'платёж',
  45: 'понятно',
  46: 'последний',
  47: 'функция',
  48: 'зайти',
  49: 'свой',
  50: 'вылетать',
  51: 'мобильный',
  52: 'стать',
  53: 'шаблон',
  54: 'приходить',
  55: 'возможность',
  56: 'право',
  57: 'делать',
  58: 'иня',
  59: 'проверка',
  

**Преобразуем значения X**

In [124]:
#преобразует каждый текст из списка в последовательность целых чисел
sequences_train = tokenizer.texts_to_sequences(text_corpus_train)
sequences_test = tokenizer.texts_to_sequences(text_corpus_test)

sequences_train[:10] #представление первый 10 текстов в виде последовательности целых чисел

[[610, 42, 775, 3539, 22, 1, 2, 15],
 [2, 181],
 [5],
 [60],
 [2],
 [127, 1],
 [37, 970, 66, 104, 30, 484, 887],
 [102, 51, 1, 690, 139, 732, 106, 1968, 53, 1969, 1625, 2],
 [463, 206, 147, 128, 99, 20, 111, 1059, 51, 9],
 [26, 1371, 888, 5, 3540, 733]]

In [125]:
word_count = len(tokenizer.index_word) + 1  #количество слов в словаре
training_length = max([len(i.split()) for i in text_corpus_train]) #максимальная длина списка токентов из тренировочного датасета
word_count, training_length 

(10293, 113)

In [126]:
#преобразуем списки последовательностей в 2D-массив (количество текстов n_samples, maxlen)
#(короткие последовательности добавляются, длинные -- усекаются. По умолчанию - с начала последовательности)
X_train = pad_sequences(sequences_train, maxlen=training_length)
X_test = pad_sequences(sequences_test, maxlen=training_length)
X_train

array([[  0,   0,   0, ...,   1,   2,  15],
       [  0,   0,   0, ...,   0,   2, 181],
       [  0,   0,   0, ...,   0,   0,   5],
       ...,
       [  0,   0,   0, ...,  27,  84, 223],
       [  0,   0,   0, ...,   0,   0,  20],
       [  0,   0,   0, ...,   0, 113,   5]])

**Преобразуем значения у**

Для подсчета функции потерь y_train и y_test должны быть представлены one-hot кодированием

In [127]:
le = LabelEncoder()
train_enc_labels = le.fit_transform(df_train['Rating']) 
test_enc_labels = le.transform(df_test['Rating'])
le.classes_

array([1, 2, 3, 4, 5], dtype=int64)

In [128]:
train_enc_labels

array([4, 4, 4, ..., 0, 4, 4], dtype=int64)

In [129]:
num_classes = 5
y_train = tf.keras.utils.to_categorical(train_enc_labels, num_classes=num_classes)
y_test = tf.keras.utils.to_categorical(test_enc_labels, num_classes=num_classes)
y_train

array([[0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1.],
       ...,
       [1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1.]], dtype=float32)

## 2. Сверточные архитектуры <a id='section_2'></a>

CONV модель Keras

In [130]:
model = Sequential()
model.add(Embedding(input_dim=word_count, output_dim=128, input_length=training_length)) 
                    #inputdim -- размер словаря, outputdim -- длина вектора, input_length -- длина входной последовательности
                    #на вход: (батч, inputlen), на выходе: (батч, inputlen, outputdim)
model.add(Conv1D(128, 3))
                   #128 -- длина 1D-фильтра, шаг -- 3
                   #на выходе ([128/3], 128)
model.add(GlobalMaxPool1D())
                    #в каждой свертке оставляет максимальный элемент
                    #на выходе ([128/3], 1)
model.add(Dense(10))
               #10-количество выходов
model.add(Dense(5))
                #num_classes = 5 -- количество выходов
model.add(Activation('softmax'))
                #преобразуем вектор в рапределение вероятностей

In [131]:
model.compile(loss='categorical_crossentropy', #y_pred должен быть распределением вероятностей, y_true -- one-hot кодированный тензор
              optimizer='adam',
              metrics=[get_f1])

In [132]:
tensorboard=TensorBoard(log_dir='./logs', write_graph=True, write_images=True)
early_stopping=EarlyStopping(monitor='val_loss')  

epochs = 20
batch_size = 512

history = model.fit(X_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[tensorboard, early_stopping])

Epoch 1/20
25/25 [==============================] - 7s 251ms/step - loss: 1.3173 - get_f1: 0.1512 - val_loss: 0.8899 - val_get_f1: 0.7005
Epoch 2/20
25/25 [==============================] - 6s 220ms/step - loss: 0.7880 - get_f1: 0.7369 - val_loss: 0.7466 - val_get_f1: 0.7468
Epoch 3/20
25/25 [==============================] - 5s 198ms/step - loss: 0.6597 - get_f1: 0.7763 - val_loss: 0.7092 - val_get_f1: 0.7585
Epoch 4/20
25/25 [==============================] - 5s 189ms/step - loss: 0.5785 - get_f1: 0.8010 - val_loss: 0.6885 - val_get_f1: 0.7635
Epoch 5/20
25/25 [==============================] - 5s 185ms/step - loss: 0.4974 - get_f1: 0.8300 - val_loss: 0.6860 - val_get_f1: 0.7719t_f1
Epoch 6/20
25/25 [==============================] - 5s 183ms/step - loss: 0.4377 - get_f1: 0.8552 - val_loss: 0.7044 - val_get_f1: 0.7699


In [133]:
score_conv = model.evaluate(X_test, y_test, batch_size=batch_size, verbose=1)
print('\n')
print('Test loss:', score_conv[0])
print('Test f1_score:', score_conv[1])

14/14 [==============================] - 1s 44ms/step - loss: 0.6734 - get_f1: 0.7796


Test loss: 0.6733518838882446
Test f1_score: 0.7796387672424316


## 3. Рекуррентные архитектуры <a id='section_3'></a>

Обучим рекуррентные нейронные сети SimpleRNN, LSTM и GRU на одних и тех же параментрах сети и сравним показатели метрики f1_score.

### 3.1. SimpleRNN <a id='section_3.1'></a>

In [134]:
model = Sequential()

model.add(Embedding(input_dim=word_count,
              input_length=training_length,
              output_dim=30,
              trainable=True,
              mask_zero=True))
model.add(Masking(mask_value=0.0))

model.add(SimpleRNN(64))  #64 -- количество ячеек, размер выходного пространства
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(5, activation='softmax'))

In [135]:
model.compile(loss='categorical_crossentropy', #y_pred должен быть распределением вероятностей, y_true -- one-hot кодированный тензор
              optimizer='adam',
              metrics=[get_f1])

In [136]:
%%time

history = model.fit(X_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[tensorboard, early_stopping])

Epoch 1/20
25/25 [==============================] - 5s 150ms/step - loss: 1.2180 - get_f1: 0.3713 - val_loss: 0.8512 - val_get_f1: 0.7264
Epoch 2/20
25/25 [==============================] - 3s 121ms/step - loss: 0.8896 - get_f1: 0.7159 - val_loss: 0.7778 - val_get_f1: 0.7356
Epoch 3/20
25/25 [==============================] - 3s 128ms/step - loss: 0.7723 - get_f1: 0.7445 - val_loss: 0.7317 - val_get_f1: 0.7493
Epoch 4/20
25/25 [==============================] - 3s 126ms/step - loss: 0.6913 - get_f1: 0.7719 - val_loss: 0.7139 - val_get_f1: 0.7542
Epoch 5/20
25/25 [==============================] - 3s 130ms/step - loss: 0.6074 - get_f1: 0.8010 - val_loss: 0.7165 - val_get_f1: 0.7609
Wall time: 17.3 s


In [137]:
score_simplernn= model.evaluate(X_test, y_test, batch_size=batch_size, verbose=1)
print('\n')
print('Test loss:', score_simplernn[0])
print('Test f1_score:', score_simplernn[1])

14/14 [==============================] - 1s 45ms/step - loss: 0.6956 - get_f1: 0.7723


Test loss: 0.6955820918083191
Test f1_score: 0.7722945213317871


### 3.2. LSTM <a id='section_3.2'></a>

In [138]:
model = Sequential()

model.add(Embedding(input_dim=word_count,
              input_length=training_length,
              output_dim=30,
              trainable=True,
              mask_zero=True))
model.add(Masking(mask_value=0.0))

model.add(LSTM(64, recurrent_dropout=0.2))  #64 -- количество ячеек, размер выходного пространства
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(5, activation='softmax'))

In [139]:
model.compile(loss='categorical_crossentropy', #y_pred должен быть распределением вероятностей, y_true -- one-hot кодированный тензор
              optimizer='adam',
              metrics=[get_f1])

In [140]:
%%time

history = model.fit(X_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[tensorboard, early_stopping])

Epoch 1/20
25/25 [==============================] - 13s 442ms/step - loss: 1.4664 - get_f1: 0.0902 - val_loss: 0.9614 - val_get_f1: 0.7226
Epoch 2/20
25/25 [==============================] - 11s 431ms/step - loss: 0.9496 - get_f1: 0.6944 - val_loss: 0.8115 - val_get_f1: 0.7320
Epoch 3/20
25/25 [==============================] - 10s 413ms/step - loss: 0.8060 - get_f1: 0.7368 - val_loss: 0.7428 - val_get_f1: 0.7412
Epoch 4/20
25/25 [==============================] - 11s 432ms/step - loss: 0.7304 - get_f1: 0.7477 - val_loss: 0.7161 - val_get_f1: 0.7489
Epoch 5/20
25/25 [==============================] - 10s 397ms/step - loss: 0.6501 - get_f1: 0.7739 - val_loss: 0.7040 - val_get_f1: 0.7540
Epoch 6/20
25/25 [==============================] - 10s 398ms/step - loss: 0.6157 - get_f1: 0.7876 - val_loss: 0.7117 - val_get_f1: 0.7629
Wall time: 1min 4s


In [141]:
score_lstm= model.evaluate(X_test, y_test, batch_size=batch_size, verbose=1)
print('\n')
print('Test loss:', score_lstm[0])
print('Test f1_score:', score_lstm[1])

14/14 [==============================] - 1s 87ms/step - loss: 0.6735 - get_f1: 0.7764


Test loss: 0.6735295653343201
Test f1_score: 0.7764195799827576


### 3.3. GRU <a id='section_3.3'></a>

In [142]:
model = Sequential()

model.add(Embedding(input_dim=word_count,
              input_length=training_length,
              output_dim=30,
              trainable=True,
              mask_zero=True))
model.add(Masking(mask_value=0.0))

model.add(GRU(64,recurrent_dropout=0.2))  #64 -- количество ячеек, размер выходного пространства
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(5, activation='softmax'))

In [143]:
model.compile(loss='categorical_crossentropy', #y_pred должен быть распределением вероятностей, y_true -- one-hot кодированный тензор
              optimizer='adam',
              metrics=[get_f1])

In [144]:
%%time

history = model.fit(X_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[tensorboard, early_stopping])

Epoch 1/20
25/25 [==============================] - 12s 388ms/step - loss: 1.5082 - get_f1: 0.0278 - val_loss: 0.9942 - val_get_f1: 0.7005
Epoch 2/20
25/25 [==============================] - 9s 346ms/step - loss: 0.9571 - get_f1: 0.7110 - val_loss: 0.8051 - val_get_f1: 0.7364
Epoch 3/20
25/25 [==============================] - 9s 350ms/step - loss: 0.7908 - get_f1: 0.7420 - val_loss: 0.7473 - val_get_f1: 0.7441
Epoch 4/20
25/25 [==============================] - 9s 349ms/step - loss: 0.7069 - get_f1: 0.7581 - val_loss: 0.7246 - val_get_f1: 0.7474
Epoch 5/20
25/25 [==============================] - 9s 349ms/step - loss: 0.6464 - get_f1: 0.7769 - val_loss: 0.7198 - val_get_f1: 0.7524
Epoch 6/20
25/25 [==============================] - 9s 353ms/step - loss: 0.5829 - get_f1: 0.7995 - val_loss: 0.7229 - val_get_f1: 0.7550
Wall time: 55.4 s


In [145]:
score_gru= model.evaluate(X_test, y_test, batch_size=batch_size, verbose=1)
print('\n')
print('Test loss:', score_gru[0])
print('Test f1_score:', score_gru[1])

14/14 [==============================] - 1s 74ms/step - loss: 0.7004 - get_f1: 0.7739


Test loss: 0.7003776431083679
Test f1_score: 0.7739042043685913


Сравним показатели метрики всех трех рекуррентных моделей

In [146]:
result_RNN = pd.DataFrame({'model': ['SimpleRNN', 'LSTM', 'GRU'], 'f1_score': [score_simplernn[1], score_lstm[1], score_gru[1]]})
result_RNN.sort_values(by='f1_score', ascending=False)

model  f1_score
1       LSTM  0.776420
2        GRU  0.773904
0  SimpleRNN  0.772295

SimpleRNN обучалась 17.3 s, LSTM --  1 m 4 s, GRU -- 55.4 s.

**Вывод:** лучшие показатели метрики f1_score у модели LSTM, правда и учится она дольше остальных. Значительно быстрее всех работает SimpleRNN, но показатели метрики у нее низкие (хотя не сильно ниже остальных).

## 4. Совместные архитектуры  <a id='section_4'></a>

Рассмотрим совместные архитектуры CNN + RNN и оценим показатели метрики таких моделей. В качестве RNN модели возьмем архитектуру LSTM.

### 4.1. CNN --> RNN <a id='section_4.1'></a>

In [178]:
model = Sequential()

model.add(Embedding(input_dim=word_count,
              input_length=training_length,
              output_dim=30,
              trainable=True,
              mask_zero=True))
model.add(Masking(mask_value=0.0))

model.add(Conv1D(128, 3))
model.add(Activation("relu"))
model.add(LSTM(64,recurrent_dropout=0.2))  #64 -- количество ячеек, размер выходного пространства
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(5, activation='softmax'))

In [179]:
model.compile(loss='categorical_crossentropy', #y_pred должен быть распределением вероятностей, y_true -- one-hot кодированный тензор
              optimizer='adam',
              metrics=[get_f1])

In [180]:
%%time

history = model.fit(X_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[tensorboard, early_stopping])

Epoch 1/20
25/25 [==============================] - 17s 609ms/step - loss: 1.3687 - get_f1: 0.1824 - val_loss: 0.8753 - val_get_f1: 0.7295
Epoch 2/20
25/25 [==============================] - 14s 574ms/step - loss: 0.8873 - get_f1: 0.7051 - val_loss: 0.7751 - val_get_f1: 0.7382
Epoch 3/20
25/25 [==============================] - 14s 574ms/step - loss: 0.7760 - get_f1: 0.7409 - val_loss: 0.7252 - val_get_f1: 0.7452
Epoch 4/20
25/25 [==============================] - 14s 576ms/step - loss: 0.6931 - get_f1: 0.7575 - val_loss: 0.7200 - val_get_f1: 0.7511
Epoch 5/20
25/25 [==============================] - 14s 561ms/step - loss: 0.6162 - get_f1: 0.7870 - val_loss: 0.7240 - val_get_f1: 0.7554
Wall time: 1min 14s


In [181]:
score_cnnrnn= model.evaluate(X_test, y_test, batch_size=batch_size, verbose=1)
print('\n')
print('Test loss:', score_cnnrnn[0])
print('Test f1_score:', score_cnnrnn[1])

14/14 [==============================] - 2s 127ms/step - loss: 0.6959 - get_f1: 0.7723


Test loss: 0.6959113478660583
Test f1_score: 0.7723208665847778


### 4.2. RNN --> CNN <a id='section_4.2'></a>

In [182]:
model = Sequential()

model.add(Embedding(input_dim=word_count,
              input_length=training_length,
              output_dim=30,
              trainable=True,
              mask_zero=True))
model.add(Masking(mask_value=0.0))

model.add(LSTM(64,recurrent_dropout=0.2, return_sequences=True))
  #return_sequences -- слоит ли возвращать последний вывод выходной последовательности или целую последовательность
model.add(Conv1D(64, 3))
model.add(GlobalMaxPool1D())
#model.add(Activation("relu"))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(5, activation='softmax'))

In [183]:
model.compile(loss='categorical_crossentropy', #y_pred должен быть распределением вероятностей, y_true -- one-hot кодированный тензор
              optimizer='adam',
              metrics=[get_f1])

In [184]:
%%time

history = model.fit(X_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[tensorboard, early_stopping])

Epoch 1/20
25/25 [==============================] - 15s 532ms/step - loss: 1.5067 - get_f1: 0.0668 - val_loss: 1.0920 - val_get_f1: 0.0000e+00
Epoch 2/20
25/25 [==============================] - 12s 501ms/step - loss: 1.1296 - get_f1: 0.5355 - val_loss: 1.0152 - val_get_f1: 0.7005
Epoch 3/20
25/25 [==============================] - 13s 499ms/step - loss: 1.0810 - get_f1: 0.6151 - val_loss: 1.0064 - val_get_f1: 0.7005
Epoch 4/20
25/25 [==============================] - 12s 497ms/step - loss: 1.0469 - get_f1: 0.6299 - val_loss: 0.8251 - val_get_f1: 0.7281
Epoch 5/20
25/25 [==============================] - 12s 493ms/step - loss: 0.8683 - get_f1: 0.7080 - val_loss: 0.7530 - val_get_f1: 0.7372
Epoch 6/20
25/25 [==============================] - 12s 497ms/step - loss: 0.7833 - get_f1: 0.7365 - val_loss: 0.7237 - val_get_f1: 0.7428
Epoch 7/20
25/25 [==============================] - 12s 492ms/step - loss: 0.7403 - get_f1: 0.7505 - val_loss: 0.7092 - val_get_f1: 0.7477
Epoch 8/20
25/25 [=====

In [185]:
score_rnncnn= model.evaluate(X_test, y_test, batch_size=batch_size, verbose=1)
print('\n')
print('Test loss:', score_rnncnn[0])
print('Test f1_score:', score_rnncnn[1])

14/14 [==============================] - 2s 109ms/step - loss: 0.6712 - get_f1: 0.7701


Test loss: 0.6712331771850586
Test f1_score: 0.7701149582862854


In [186]:
result_CNN_RNN = pd.DataFrame({'model': ['CNN_RNN', 'RNN_CNN'], 'f1_score': [score_cnnrnn[1], score_rnncnn[1]]})
result_CNN_RNN.sort_values(by='f1_score', ascending=False)

model  f1_score
0  CNN_RNN  0.772321
1  RNN_CNN  0.770115

Модель CNN_RNN обучалась 1 m 14 s, модель RNN_CNN -- 1 m  54 s.

**Вывод:** лучшие показатели метрики f1_score у модели CNN + LSTM и обучается она быстрее LSTM + CNN.

## 5. Выводы  <a id='section_5'></a>

Оценим сводную таблицу метрик всех рассмотренных в задании моделей.

In [196]:
result_CNN = pd.DataFrame({'model': 'CNN', 'f1_score': [score_conv[1]]})
result_models = pd.concat([result_CNN, result_RNN, result_CNN_RNN], ignore_index=True)
result_models.sort_values(by='f1_score', ascending=False)

model  f1_score
0        CNN  0.779639
2       LSTM  0.776420
3        GRU  0.773904
4    CNN_RNN  0.772321
1  SimpleRNN  0.772295
5    RNN_CNN  0.770115

**Вывод:** лучшие показатели метрики f1_score у модели CNN. 